In [1]:
# Purchasing Analysis (Total)
import pandas as pd
csvpath = "./purchase_data.csv"
purchase_data = pd.read_csv(csvpath)

purchase_data = purchase_data.rename(columns={"SN":"Player Name"})
purchase_data = purchase_data[["Item ID","Player Name","Gender","Age","Item Name","Price"]]

unique_players = len(purchase_data["Player Name"].unique())
unique_items = len(purchase_data["Item Name"].unique())
average_price = round(purchase_data["Price"].mean(),2)
total_revenue = round(purchase_data["Price"].sum(),2)
number_of_purchases = len(purchase_data)
print(f"There are {unique_players} total players.")

purchasing_analysis = {"Number of Unique Items":[unique_items],
                     "Average Price":[average_price],
                      "Number of Purchases":[number_of_purchases],
                      "Total Revenue":[total_revenue]}

purchasing_analysis_df = pd.DataFrame(purchasing_analysis)
purchasing_analysis_df["Average Price"] = purchasing_analysis_df["Average Price"].map("${0:.2f}".format)
purchasing_analysis_df["Total Revenue"] = purchasing_analysis_df["Total Revenue"].map("${0:,.2f}".format)

purchasing_analysis_df

There are 576 total players.


,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,179,$3.05,780,"$2,379.77"


In [2]:
# Gender Demographics
gender_data = purchase_data.iloc[:,1:3]
gender_data = gender_data.drop_duplicates("Player Name")
gender_data_grouped = gender_data.groupby(["Gender"])
gender_count = gender_data_grouped.count()
gender = gender_count

female_percent = 100 * (gender.iloc[0,0]/(unique_players))
male_percent = 100 * (gender.iloc[1,0]/(unique_players))
other_percent = 100 * (gender.iloc[2,0]/(unique_players))

gender["Percentage of Players"] = [female_percent,male_percent, other_percent]
gender = gender.rename(columns={"Player Name":"Total Count"})
gender["Percentage of Players"].dtype
gender["Percentage of Players"] = pd.to_numeric(gender["Percentage of Players"])
gender["Percentage of Players"] = gender["Percentage of Players"].map("{0:.2f}%".format)
gender_sorted = gender.sort_values(["Total Count"],ascending=False)
gender_sorted

,Total Count,Percentage of Players
Gender,,
Male,484,84.03%
Female,81,14.06%
Other / Non-Disclosed,11,1.91%


In [3]:
# Purchasing Anaylsis (Gender)
purchase_data_grouped = purchase_data.groupby(["Gender"])
purchase_count = pd.DataFrame(purchase_data_grouped.count())
purchase_count = purchase_count[["Player Name"]]
purchase_count = purchase_count.rename(columns={"Player Name":"Purchase Count"})
purchase_count = purchase_count.sort_values(["Purchase Count"], ascending=False)

average_purchase_price = purchase_data_grouped.mean()
average_purchase_price = average_purchase_price[["Price"]]
average_purchase_price["Price"] = average_purchase_price["Price"].map("${0:.2f}".format)
average_purchase_price = average_purchase_price.rename(columns={"Price":"Average Purchase Price"})
average_purchase_price = average_purchase_price.sort_values(["Average Purchase Price"])

total_purchase_value = purchase_data_grouped.sum()
total_purchase_value = total_purchase_value[["Price"]]
total_purchase_float = {}
total_purchase_float["Price"] = pd.to_numeric(total_purchase_value["Price"])
total_purchase_value["Price"] = total_purchase_value["Price"].map("${0:.2f}".format)
total_purchase_value = total_purchase_value.rename(columns={"Price":"Total Purchase Value"})
total_purchase_value = total_purchase_value.sort_values(["Total Purchase Value"],ascending=True)

total_purchase_series = total_purchase_value
total_purchase_series.columns = ["Total Purchase Value"]
total_purchase_series = total_purchase_series["Total Purchase Value"]
male_total_purchase_value = total_purchase_series[0]
female_total_purchase_value = total_purchase_series[1]
other_total_purchase_value = total_purchase_series[2]

average_purchases = pd.DataFrame(total_purchase_float["Price"]/gender_sorted["Total Count"])
average_purchases = average_purchases.rename(columns={0:"Avg Total Purchase per Person"})
average_purchases["Avg Total Purchase per Person"] = average_purchases["Avg Total Purchase per Person"].map("${0:.2f}".format)

merge_table = pd.merge(purchase_count, average_purchase_price,on="Gender")
merge_table = pd.merge(merge_table,total_purchase_value, on="Gender")
merge_table = pd.merge(merge_table,average_purchases,on="Gender")
merge_table

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Gender,,,,
Male,652,$3.02,$1967.64,$4.07
Female,113,$3.20,$361.94,$4.47
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


In [4]:
# Age Demographics
bins=[0,9,14,19,24,29,34,39,1000]
group_names=[">10","10-14","15-19","20-24","25-29","30-34","35-39","40+"]

age_data = purchase_data.drop_duplicates("Player Name")
age_data ["Age Group"] = pd.cut(age_data["Age"], bins, labels=group_names)
age_data_grouped = age_data.groupby(["Age Group"])
age_group_total_count = age_data_grouped.count()
age_group_total_count = age_group_total_count[["Player Name"]]
age_group_total_count = age_group_total_count.rename(columns={"Player Name":"Total Count"})

age_group_total_count["Percentage of Players"] = 100* (age_group_total_count["Total Count"]/unique_players)
age_group_total_count["Percentage of Players"] = age_group_total_count["Percentage of Players"].map("{0:.2f}%".format)
age_group_total_count

D:\Users\9csic\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,Total Count,Percentage of Players
Age Group,,
>10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


In [5]:
# Purchasing Analysis (Age)
age_purchase_data = purchase_data
age_purchase_data ["Age Group"] = pd.cut(age_purchase_data["Age"], bins, labels=group_names)
age_purchase_data = age_purchase_data.groupby(["Age Group"])

age_average_purchase_price = age_purchase_data["Price"].mean()
age_purchase_count = age_purchase_data["Player Name"].count()
age_total_purchase = age_purchase_data["Price"].sum()


summary_table = {}
summary_table["Purchase Count"] = age_purchase_count
summary_table["Average Purchase Price"] = age_average_purchase_price
summary_table["Total Purchase Value"] = age_total_purchase
summary_table["Avg Total Purchase per Person"] = age_total_purchase/age_data_grouped["Player Name"].count()

summary_table = pd.DataFrame(summary_table)

summary_table["Average Purchase Price"] = summary_table["Average Purchase Price"].map("${0:.2f}".format)
summary_table["Total Purchase Value"] = summary_table["Total Purchase Value"].map("${0:,.2f}".format)
summary_table["Avg Total Purchase per Person"] = summary_table["Avg Total Purchase per Person"].map("${0:,.2f}".format)

summary_table

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Age Group,,,,
>10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,"$1,114.06",$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


In [6]:
# Top Spenders
top_spender_data = purchase_data.groupby(["Player Name"])
top_spender_purchase_count = top_spender_data["Player Name"].count()

top_spender_total_purchase_value = top_spender_data["Price"].sum()

top_spender_summary = {}
top_spender_summary["Purchase Count"] = top_spender_purchase_count
top_spender_summary["Average Purchase Price"] = top_spender_total_purchase_value/top_spender_purchase_count
top_spender_summary["Total Purchase Value"] = top_spender_total_purchase_value

top_spender_df = pd.DataFrame(top_spender_summary)

top_spender_df["Average Purchase Price"] = top_spender_df["Average Purchase Price"].map("${0:,.2f}".format)
top_spender_df["Total Purchase Value"] = pd.to_numeric(top_spender_df["Total Purchase Value"])
top_spender_df = top_spender_df.sort_values(["Total Purchase Value"],ascending=False)
top_spender_df["Total Purchase Value"] = top_spender_df["Total Purchase Value"].map("${0:,.2f}".format)
top_spender_df.head()

,Purchase Count,Average Purchase Price,Total Purchase Value
Player Name,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


In [7]:
# Most Popular Items
item_data = purchase_data.groupby(["Item ID","Item Name"])
item_count = item_data["Player Name"].count()
item_price = item_data["Price"].mean()

most_popular_items = {}
most_popular_items["Purchase Count"] = item_count
most_popular_items["Item Price"] = round(item_price,2)
most_popular_items["Total Purchase Value"] = item_count * item_price
most_popular_items_df = pd.DataFrame(most_popular_items)
most_popular_items_df = most_popular_items_df.sort_values(["Purchase Count"],ascending=False)
most_popular_items_df["Item Price"] = most_popular_items_df["Item Price"].map("${0:.2f}".format)
most_popular_items_df["Total Purchase Value"] = pd.to_numeric(most_popular_items_df["Total Purchase Value"])
most_popular_items_df["Total Purchase Value"] = most_popular_items_df["Total Purchase Value"].map("${0:.2f}".format)
most_popular_items_df.head()

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
132,Persuasion,9,$3.22,$28.99
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77


In [8]:
# Most Profitable Items
most_profitable_items = most_popular_items

most_profitable_items_df = pd.DataFrame(most_profitable_items)
most_profitable_items_df["Total Purchase Value"] = pd.to_numeric(most_profitable_items_df["Total Purchase Value"])
most_profitable_items_df = most_profitable_items_df.sort_values(["Total Purchase Value"],ascending=False)
most_profitable_items_df["Item Price"] = most_profitable_items_df["Item Price"].map("${0:.2f}".format)
most_profitable_items_df["Total Purchase Value"] = most_profitable_items_df["Total Purchase Value"].map("${0:.2f}".format)
most_profitable_items_df.head()

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
103,Singed Scalpel,8,$4.35,$34.80
